![sql](./img/sql-logo.jpg)

# A beginner's guide to databases, SQL, & using them with `pandas`

**Scenario:** You are a data analyst for the Homeland Security, trying to create reports on the active airports world wide. The data you need to access is in a SQL database. You need to be able to query for the data in a database!

## Learning goals:
- Goal 1: Summarize the use case for sql in the data science skill set
- Goal 2: Define key sql terminology
- Goal 3: Get information about DB schema and table structure
- Goal 4: Use basic SQL commands:
    - Construct SQL queries
    - Use `JOIN` to merge tables along logical columns
    - Grouping Data with SQL, inlcuding `HAVING`
- Goal 5: Convert SQL to pandas

## Goal 1: Summarize

To date the data we've seen has looked like [this.](https://data.austintexas.gov/Health-and-Community-Services/Austin-Animal-Center-Intakes/wter-evkm)

That is how we need data to look to run analysis and build models.<br>
But it doesn't _live_ there in it's native state.

[Netflix has a great article](https://medium.com/netflix-techblog/notebook-innovation-591ee3221233) describing three different data roles at their company, their different needs, and their toolsets.

![netflix](./img/netflix-data-roles.jpeg)

Examining that graphic, SQL shows up as one of the tools of the _Data Engineer_ 

Data Engineers provide the essential data architecture services that make data science possible.

![hierarchy](./img/ai-hierachy.png)

[Source: Monica Rogati’s fantastic Medium post “The AI Hierarchy of Needs”
](https://hackernoon.com/the-ai-hierarchy-of-needs-18f111fcc007)

## What does it mean to **Engineer Data**?

Let's start with a basic scenario:<br>
You are HR. You have *no* current database. 
How would you set it up?

Data needs considerations:
- What you want to store
- What "views" you anticipate wanting in the future

Structure considerations:
- Speed of retrieval
- How much data you are accessing
- How much you are storing

![etl](./img/etl.png)

[img source: Jeff Hammerbacher’s slide from UC Berkeley CS 194 course ](https://bcourses.berkeley.edu/courses/1377158/pages/cs-194-16-introduction-to-data-science-fall-2015)

### What is a Relational Database? 

![rdb](./img/relational-dbms-model.png)
[reference for image ](https://www.studytonight.com/dbms/database-model.php)

### POPULAR RDBMS

- SQLite
- MySQL
- PostgreSql
- Oracle DB
- SQL Server

***
### SQL

[History of SQL](https://www.businessnewsdaily.com/5804-what-is-sql.html)

> Future users of large data banks must be protected from having to know how the data is organized in the machine (the internal representation). A prompting service which supplies such information is not a satisfactory solution. Activities of users at terminals and most application programs should remain unaffected when the internal representation of data is changed and even when some aspects of the external representation are changed. Changes in data representation will often be needed as a result of changes in query, update, and report traffic and natural growth in the types of stored information.
.
.
.<br>
<br>
In Section 1 a relational model of data is proposed as a basis for protecting users of formatted data systems from the potentially disruptive changes in data representation caused by growth in the data bank and changes in traffic.

[The original SQL paper from the 1970s.](https://www.seas.upenn.edu/~zives/03f/cis550/codd.pdf)

***
## Goal 2: Database terminology

### Relational Database Schema

![schema](./img/MySQL_Schema_Music_Example.png)

[source of image](https://database.guide/what-is-a-database-schema/)

***
### SQLite

<img src="./img/SQLite-Python.jpg" width=500>

"SQLite is a C library that provides a lightweight disk-based database that doesn’t require a separate server process and allows accessing the database using a nonstandard variant of the SQL query language. Some applications can use SQLite for internal data storage. It’s also possible to prototype an application using SQLite and then port the code to a larger database such as PostgreSQL or Oracle." - [sqlite documentation](https://docs.python.org/2/library/sqlite3.html)



## But what about connecting to database servers?

To connect to other database types there are many different libraries:
- Oracle: [cx_Oracle](https://oracle.github.io/python-cx_Oracle/)
- MySQL: [MySQL-python](https://github.com/farcepest/MySQLdb1)
- PostgreSQL: [Psycopg2](http://initd.org/psycopg/docs/)
- Microsoft SQL Server: [pymssql](http://www.pymssql.org/en/stable/)

These all implement [PEP 249: DB API v2](https://www.python.org/dev/peps/pep-0249/)

### The Structure of a SQL Query

<img src='./img/sql_statement.jpg'/>

***
## Goal 3: Get going with sqlite!

In [ ]:
import sqlite3

#### Load a DB object with `connect` and `cursor`

**Connection object** is your connection to the database, close that when you're done talking to the database all together. 

**Cursor object** is an iterator over a result set from a query. 

In [ ]:
con = sqlite3.connect('data/flights.db')
cursor = con.cursor()

#### Use sqlite_master to find all the tables in the schema
Get the schema of a database from a db in sqlite

The `.excecute()` function will execute a sql statement and our `fetchall()` function will give us a list of all rows matching the execute statement.

In [ ]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

In [ ]:
cursor.description

In [ ]:
cursor.execute("SELECT name, type FROM sqlite_master;")
print(cursor.fetchall())

#### Get information about one table

**A note about** `execute`<br>
Each time you use it, you reset the value of cursor

In [ ]:
cursor.execute("SELECT * FROM airports as a;").fetchone()

In [ ]:
cursor.fetchone()

In [ ]:
cursor.fetchmany(4)

#### Use description

In [ ]:
cursor.description

#### Or use `Pragma`
`Pragma` tool [link here](https://www.sqlite.org/pragma.html#pragma_table_info)

**output**<br>
`(column id, column name, data type, whether or not the column can be NULL, and the default value for the column)`

In [ ]:
cursor.execute("PRAGMA table_info(airports);")
info = cursor.fetchall()
print(*info, sep = "\n") 

#### Making fetch happen

`.fetchall()` is how you get the query results out of the object.

You can also `.fetchone()` or `.fetchmany()`

**Task:** Get the descriptive data for airlines and routes tables

In [ ]:
cursor.execute("PRAGMA table_info(airlines);")
info = cursor.fetchall()
print(*info, sep = "\n")

In [ ]:
cursor.execute("PRAGMA table_info(routes);")
info = cursor.fetchall()
print(*info, sep = "\n")

***
## Goal 4: Use basic SQL commands 
- Construct SQL queries
- Use JOIN to merge tables along logical columns
- Grouping Data with SQL

### Construct SQL queries

**Select**

**From**

**Where**

**Order by**

**Limit**

#### Options for each:

**Select**:  `distinct`, using `as` to rename columns, single number aggregates

**From:** also uses aliasing with `as`

**Where**: equals is only one `=`, `between`, `in`, wildcards `%`, `AND`, `OR`, `NOT`

**Order by**: `asc` and `desc`

**Limit**:  #

**Task**: 
- Select only active airlines in the UK from the airlines table
- Select the unique list of countries with airports

In [ ]:
cursor.execute("""
SELECT name
FROM airlines
WHERE active = 'Y'
  AND country = 'United Kingdom';
""")
cursor.fetchall()

In [ ]:
cursor.execute(
"""
SELECT DISTINCT country
FROM airports;
"""
).fetchall()

### SQL Joins

SQL joins can be used to both **add** data to a table and **remove** data from a table. 

<img src="./img/venn.png" width=550>

**Task** Write a query that will join the latitude and longitude data from the airports table to the information on the routes table

In [ ]:
cursor.execute("""
SELECT *
FROM routes as r
JOIN airports as a
ON a.code=r.dest;
""").fetchmany(5)

### Grouping statements

Combines `select` and `group by` when you want aggregates by values

`select` `min(x)` ... `max()`, `sum()`, etc

`group by x`

### Let's answer some questions

- Which countries have the highest amount of active airlines?
- Which countries have the highest amount of inactive airlines?

In [ ]:
# Which countries have the highest amount of active airports?
cursor.execute("""
SELECT COUNT() as num, country
FROM airlines
WHERE active='Y'
GROUP BY country
HAVING num > 10
ORDER BY num DESC
LIMIT 30;
""")
print(*cursor.fetchall(), sep='\n')

In [ ]:
# Which countries have the highest amount of inactive airports?
cursor.execute("""
SELECT COUNT() num, country
FROM airlines
WHERE active='N'
GROUP BY country
HAVING num > 10
ORDER BY num DESC
LIMIT 30;
""")
print(*cursor.fetchall(), sep='\n')

### Your turn!

In your group write some code to answer the following question:

- What are the number of airports for each time zone?  Each row should have a number of airports and a time zone.

In [ ]:
# What about airports by timezones?
cursor.execute("""
-- your code here;""")
print(*cursor.fetchall(), sep='\n')

In [ ]:
# It's always a good idea to close our connections when we're done
cursor.close()
con.close()

### Using DB Browser for SQLite 

![](https://sqlitebrowser.org/images/screenshot.png)

DB Browser for SQLite is a high quality, visual, open source tool to create, design, and edit database files compatible with SQLite.

DB4S is for users and developers who want to create, search, and edit databases. DB4S uses a familiar spreadsheet-like interface, and complicated SQL commands do not have to be learned.

#### Let's practice a few queries in DB Browser for SQLite


## Goal 5: Transfering from sqlite to pandas

In [ ]:
import pandas as pd
pd_con = sqlite3.connect("./data/flights.db")
df = pd.read_sql_query("select * from airports limit 5;", pd_con)
df

**Task**: 
Convert one of the earlier queries in the lesson to a pandas data frame

In [ ]:
# Which countries have the highest amount of active airports?
sql = '''
SELECT COUNT() num, country
FROM airlines
WHERE active='Y'
GROUP BY country
HAVING num > 10
ORDER BY num DESC
LIMIT 30;
'''
pd.read_sql_query(sql, pd_con)

In [ ]:
pd_con.close()

## Your Turn!

The [Chinook database](https://github.com/lerocha/chinook-database) is a sample database, representing a digital media store.

You need to create a query that can rank tracks in term of popularity.

The name of the database is `Chinook_Sqlite.sqlite`

In your group, answer the following questions about the data:
- What are the max and min dates in the Invoice table?
- What tables would you need to answer "what is your most popular track?"
- What values from each table?

In [ ]:
conn = sqlite3.connect('data/Chinook_Sqlite.sqlite')
cur = conn.cursor()

In [ ]:
# What are the max and min dates in the Invoice table?
sql = '''
-- YOUR CODE HERE
'''
pd.read_sql_query(sql, conn)

In [ ]:
# What tables would you need to answer "what is your most popular track?"


In [ ]:
# What values would you need from each table?

In [ ]:
# Put it all together:
# You need to create a query that can rank tracks in term of popularity.

sql = '''
-- YOUR CODE HERE
'''
pd.read_sql_query(sql, conn)

In [ ]:
# Advanced: get the artist who sang the song!
sql = '''
-- YOUR CODE HERE
'''
pd.read_sql_query(sql, conn)